<a href="https://colab.research.google.com/github/DeepManuPy/MultiLayer-Perceptrons/blob/master/HE_Comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
seed = 7
np.random.seed(seed)

In [0]:
import pandas as pd

df = pd.read_csv("train.csv")

dataset = df.values
df = df.sample(frac=1).reset_index(drop=True)

to_drop = ['ID','DateTime','HiDir','WindDir']
df = df.drop(columns = to_drop)

feature_cols = ['TempOut','HiTemp','LowTemp','OutHum','DewPt','WindSpeed',
               'WindRun','HiSpeed','WindChill','HeatIndex','THWIndex','Bar',
               'Rain','RainRate','HeatDD','CoolDD','InTemp','InDew','InDew',
               'InHeat','InEMC','InAirDensity','WindSamp','WindTx','ISSRecpt',
               'ArcInt']
target_cols = ['PA','PB','PC','PD','PE','PF','PG']

X = df[feature_cols].values
y = df[target_cols].values

In [0]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
X[] = scale.fit_transform(X)

In [0]:
from keras import backend as K
def root_mean_squared_error(y_true,y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true),axis=-1))

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization
from tensorflow.keras.optimizers import Adam

def baseline_model():
  model = Sequential()
  model.add(Dense(26,input_dim=26,activation='relu'))
  model.add(Dense(13,activation='relu'))
  model.add(Dense(7,activation='linear'))
  model.compile(optimizer='adam',loss=root_mean_squared_error,metrics=['accuracy'])
  return model
def model_1():
  model = Sequential()
  model.add(Dense(26,input_dim=26,activation='relu'))
  model.add(Dense(18,activation='relu'))
  model.add(Dense(7,activation='linear'))
  model.compile(optimizer='adam',loss=root_mean_squared_error,metrics=['accuracy'])
  return model
  

In [68]:
model = model_1()

model.fit(X,y,epochs=200,batch_size=16,validation_split=0.1,verbose=1)

Train on 36000 samples, validate on 4000 samples
Epoch 1/200
36000/36000 [==============================] - 4s 101us/sample - loss: 162.8096 - acc: 0.9886 - val_loss: 155.6426 - val_acc: 0.9998
Epoch 2/200
36000/36000 [==============================] - 3s 89us/sample - loss: 159.9343 - acc: 0.9825 - val_loss: 154.6039 - val_acc: 0.9540
Epoch 3/200
36000/36000 [==============================] - 3s 89us/sample - loss: 159.0030 - acc: 0.9720 - val_loss: 154.6341 - val_acc: 0.8727
Epoch 4/200
36000/36000 [==============================] - 3s 88us/sample - loss: 157.9844 - acc: 0.9644 - val_loss: 152.3745 - val_acc: 0.9650
Epoch 5/200
36000/36000 [==============================] - 3s 89us/sample - loss: 156.8801 - acc: 0.9621 - val_loss: 151.6519 - val_acc: 0.9525
Epoch 6/200
36000/36000 [==============================] - 3s 90us/sample - loss: 155.9533 - acc: 0.9580 - val_loss: 151.2964 - val_acc: 0.9178
Epoch 7/200
36000/36000 [==============================] - 3s 89us/sample - loss: 155.

In [0]:
model.save_weights("model-1.hdf5")

In [0]:
ds = pd.read_csv("test.csv")

#dataset = df.values
ds = ds.sample(frac=1).reset_index(drop=True)

to_drop = ['ID','DateTime','HiDir','WindDir']
ds = ds.drop(columns = to_drop)

feature_cols = ['TempOut','HiTemp','LowTemp','OutHum','DewPt','WindSpeed',
               'WindRun','HiSpeed','WindChill','HeatIndex','THWIndex','Bar',
               'Rain','RainRate','HeatDD','CoolDD','InTemp','InDew','InDew',
               'InHeat','InEMC','InAirDensity','WindSamp','WindTx','ISSRecpt',
               'ArcInt']

X_test = ds[feature_cols].values

model.load_weights("model-1.hdf5")

predicted = model.predict(X_test)

vals = np.round(predicted)
range =np.arange(40001,62447)

pa = vals[:,0].tolist()
pb = vals[:,1].tolist()
pc = vals[:,2].tolist()
pda = vals[:,3].tolist()
pe = vals[:,4].tolist()
pf = vals[:,5].tolist()
pg = vals[:,6].tolist()

with open("output-model-1.csv","w") as f:
  f.write("ID,PA,PB,PC,PD,PE,PF,PG\n")
  for x,y1,y2,y3,y4,y5,y6,y7 in zip(range,pa,pb,pc,pda,pe,pf,pg):
    f.write("PR{},{},{},{},{},{},{},{}\n".format(x,y1,y2,y3,y4,y5,y6,y7))
f.close()